# LM based on convolutions

First step, reimplement https://arxiv.org/abs/1612.08083

In [19]:
# Import all needed stuff
import tensorflow as tf
import numpy as np
from IPython.display import display, HTML, clear_output
import functools
import random
from collections import namedtuple

In [20]:
print(tf.__version__)

1.8.0


In [7]:
# model parameters
params = tf.contrib.training.HParams(
    # data params
    max_string_length = 128,
    
    # training params
    batch_size = 256,
    shuffle_buffer = 100000,
    num_epochs = 10,
    
    # Model parameters
    ## Embedding
    emedding_size = 64,
    embedding_range = 0.1,
)

In [40]:
# Converts a string of text into a vector of char codes.
def _text_to_codes(text_string):
    in_array = np.array([ord(c) for c in text_string], dtype=np.uint8)
    return in_array

def convert_example(example, params=params):
    example=example.decode("utf-8")
    example = example.strip()[:params.max_string_length]
    split_pos = (len(example)*4)//5
    context = example[:split_pos]
    prediction = example[split_pos]
    return (_text_to_codes(context), _text_to_codes(prediction))


print(convert_example(b"A quick and brown fox", params))

(array([ 65,  32, 113, 117, 105,  99, 107,  32,  97, 110, 100,  32,  98,
       114, 111, 119], dtype=uint8), array([110], dtype=uint8))


In [41]:
def generateDataset(params):
    """Creates a dataset with char encoded codes"""
    dataset = tf.data.TextLineDataset("big_set.en")
    dataset = dataset.repeat(params.num_epochs)
    dataset = dataset.shuffle(params.shuffle_buffer)
    dataset = dataset.map(lambda text_string:
                          tf.py_func(functools.partial(convert_example, params=params),
                                     [text_string], [tf.uint8, tf.uint8]))
    dataset = dataset.padded_batch(params.batch_size,
                                   padded_shapes=([(params.max_string_length*4)//5], [1]))
    training_iterator = dataset.make_initializable_iterator()
    next_element = training_iterator.get_next()
    return (training_iterator, next_element)
    
    

In [ ]:
def EmbedInput(input, params, reuse_vars):
    with tf.variable_scope("embedding", reuse=reuse_vars):
        embedding = tf.get_variable("char_embedding", shape=[256,params.embedding_size],
                                   dtype=tf.float32,
                                   initializer=tf.initializers.random_uniform(minval=-params.embedding_range,
                                                                             maxval=params.embedding_range))
        embedded = tf.nn.embedding_lookup(embedding, input)
        return embedded

def GatedConvolution(input, params, reuse_vars, level_descr):
    with tf.variable_scope("gcl_%d" % level, reuse=reuse_vars):
        conv1_filters = tf.get_variable("conv1_filter", shape=[])
        conv1 = tf.nn.conv1d(input, conv1_filters, stride=1, padding="SAME")
        fully_connected1 = tf.contrib.layers.fully_connected(conv1, num_outputs=,activation_fn=None,)
    
    
    
def EncodeContext(context, params, reuse_vars):
    """Encodes context using gated convolutions https://arxiv.org/abs/1612.08083"""
    prev_values = EmbedInput(context, params, reuse_vars)
    for level_descr in params.level_descriptions:
        prev_values = Gatted
        
    

In [42]:
with tf.Graph().as_default():
    training_iterator, next_element = generateDataset(params)
    training_flag = tf.placeholder(tf.bool, shape=(), name="training_flag")
    global_step = tf.Variable(0, trainable=False, name="global_step")
    context, prediction = next_element
    encoded = EncodeContext(context, params)
    scaffold = tf.train.Scaffold(
        local_init_op=tf.group(tf.local_variables_initializer(),training_iterator.initializer),
        init_op=tf.global_variables_initializer())
    
    with tf.train.MonitoredTrainingSession(checkpoint_dir="./checkpoint",scaffold=scaffold,
                                           save_checkpoint_secs=120) as sess:
        print("Start training")
        while not sess.should_stop():
            training_o = sess.run(next_element)
            print(training_o)


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./checkpoint/model.ckpt-0
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Start training
INFO:tensorflow:Saving checkpoints for 0 into ./checkpoint/model.ckpt.
(array([[ 80, 114,  97, ...,   0,   0,   0],
       [105, 115,  32, ...,   0,   0,   0],
       [ 99, 114, 105, ...,   0,   0,   0],
       ...,
       [100, 101,  97, ...,   0,   0,   0],
       [ 97, 115, 116, ...,   0,   0,   0],
       [ 68,  97, 110, ...,   0,   0,   0]], dtype=uint8), array([[116],
       [ 98],
       [116],
       [ 32],
       [111],
       [109],
       [109],
       [ 45],
       [100],
       [ 32],
       [ 97],
       [105],
       [108],
       [111],
       [108],
       [ 32],
       [101],
       [104],
       [109],
       [ 83],
       [115],
       [100],
       [100],
       [ 32],
       [ 66],
       [ 32],
       [115],
       [116]

(array([[ 50,  48,  48, ...,   0,   0,   0],
       [ 99, 111, 109, ...,   0,   0,   0],
       [ 97, 110, 100, ...,   0,   0,   0],
       ...,
       [ 97, 100, 109, ...,   0,   0,   0],
       [116, 111,  32, ...,   0,   0,   0],
       [116, 104, 101, ...,   0,   0,   0]], dtype=uint8), array([[117],
       [104],
       [119],
       [116],
       [101],
       [105],
       [100],
       [114],
       [110],
       [ 67],
       [116],
       [ 97],
       [100],
       [110],
       [ 32],
       [105],
       [111],
       [101],
       [100],
       [ 57],
       [ 32],
       [101],
       [120],
       [108],
       [ 97],
       [ 32],
       [ 32],
       [ 97],
       [ 32],
       [ 72],
       [104],
       [100],
       [ 97],
       [ 32],
       [111],
       [101],
       [ 32],
       [105],
       [ 97],
       [ 32],
       [ 32],
       [110],
       [110],
       [ 32],
       [115],
       [116],
       [ 32],
       [ 73],
       [110],
       [101],
       [

(array([[ 73, 110,  32, ...,   0,   0,   0],
       [ 84, 111, 117, ...,   0,   0,   0],
       [ 97, 110, 100, ...,   0,   0,   0],
       ...,
       [100, 105, 100, ...,   0,   0,   0],
       [105, 115,  32, ...,   0,   0,   0],
       [115, 105, 109, ...,   0,   0,   0]], dtype=uint8), array([[101],
       [104],
       [112],
       [115],
       [116],
       [111],
       [101],
       [ 99],
       [ 32],
       [115],
       [ 32],
       [ 32],
       [114],
       [114],
       [104],
       [110],
       [114],
       [114],
       [115],
       [ 49],
       [104],
       [121],
       [104],
       [101],
       [114],
       [ 32],
       [ 74],
       [100],
       [116],
       [105],
       [ 32],
       [ 32],
       [110],
       [101],
       [114],
       [108],
       [111],
       [109],
       [110],
       [111],
       [101],
       [112],
       [ 32],
       [ 98],
       [104],
       [ 32],
       [ 32],
       [104],
       [ 32],
       [117],
       [

(array([[111, 102,  32, ...,   0,   0,   0],
       [108,  97, 121, ...,   0,   0,   0],
       [ 99, 111, 110, ...,   0,   0,   0],
       ...,
       [ 40,  99, 111, ...,   0,   0,   0],
       [ 97, 110, 100, ...,   0,   0,   0],
       [ 34,  84, 105, ...,   0,   0,   0]], dtype=uint8), array([[ 85],
       [100],
       [110],
       [ 97],
       [ 97],
       [ 99],
       [104],
       [116],
       [101],
       [ 82],
       [122],
       [ 32],
       [110],
       [ 32],
       [104],
       [108],
       [111],
       [118],
       [103],
       [ 97],
       [ 32],
       [108],
       [111],
       [121],
       [ 45],
       [116],
       [ 97],
       [103],
       [112],
       [101],
       [115],
       [ 40],
       [104],
       [ 41],
       [105],
       [110],
       [114],
       [116],
       [115],
       [110],
       [101],
       [ 32],
       [100],
       [108],
       [111],
       [ 98],
       [ 32],
       [111],
       [101],
       [101],
       [

(array([[102, 105, 114, ...,   0,   0,   0],
       [116, 104, 101, ...,   0,   0,   0],
       [104, 101,  32, ...,   0,   0,   0],
       ...,
       [101, 118, 101, ...,   0,   0,   0],
       [115, 117, 112, ...,   0,   0,   0],
       [110, 101, 101, ...,   0,   0,   0]], dtype=uint8), array([[100],
       [ 97],
       [114],
       [ 32],
       [ 65],
       [117],
       [ 97],
       [100],
       [ 99],
       [109],
       [114],
       [101],
       [ 99],
       [114],
       [116],
       [109],
       [ 97],
       [101],
       [ 79],
       [115],
       [110],
       [105],
       [ 32],
       [109],
       [110],
       [110],
       [116],
       [116],
       [116],
       [ 99],
       [115],
       [115],
       [100],
       [ 65],
       [101],
       [103],
       [108],
       [ 32],
       [105],
       [119],
       [111],
       [100],
       [108],
       [ 99],
       [111],
       [101],
       [114],
       [119],
       [115],
       [ 97],
       [

...

(array([[116, 104, 101, ...,   0,   0,   0],
       [ 73, 110,  32, ...,   0,   0,   0],
       [ 66,  97, 115, ...,   0,   0,   0],
       ...,
       [104,  97, 118, ...,   0,   0,   0],
       [112, 114, 111, ...,   0,   0,   0],
       [ 97, 110,  32, ...,   0,   0,   0]], dtype=uint8), array([[ 32],
       [ 32],
       [ 32],
       [ 32],
       [ 99],
       [ 97],
       [114],
       [112],
       [112],
       [ 32],
       [111],
       [ 32],
       [ 32],
       [116],
       [115],
       [101],
       [ 55],
       [104],
       [101],
       [ 39],
       [117],
       [101],
       [ 85],
       [100],
       [ 32],
       [103],
       [121],
       [ 32],
       [ 65],
       [110],
       [ 34],
       [111],
       [ 98],
       [108],
       [ 75],
       [101],
       [104],
       [111],
       [105],
       [101],
       [ 32],
       [ 32],
       [108],
       [100],
       [100],
       [115],
       [ 32],
       [109],
       [101],
       [ 84],
       [

(array([[105, 115,  32, ...,   0,   0,   0],
       [ 99, 111, 108, ...,   0,   0,   0],
       [ 98, 121,  32, ...,   0,   0,   0],
       ...,
       [ 97, 115,  32, ...,   0,   0,   0],
       [109, 101,  99, ...,   0,   0,   0],
       [ 84, 104, 101, ...,   0,   0,   0]], dtype=uint8), array([[111],
       [116],
       [102],
       [100],
       [115],
       [101],
       [114],
       [103],
       [ 97],
       [117],
       [ 57],
       [109],
       [102],
       [100],
       [110],
       [ 97],
       [111],
       [111],
       [111],
       [112],
       [ 32],
       [111],
       [ 32],
       [ 32],
       [104],
       [116],
       [110],
       [103],
       [ 50],
       [111],
       [117],
       [115],
       [ 41],
       [115],
       [116],
       [111],
       [116],
       [107],
       [115],
       [114],
       [115],
       [ 32],
       [ 99],
       [111],
       [105],
       [110],
       [102],
       [101],
       [ 97],
       [ 32],
       [

(array([[ 97,  32, 116, ...,   0,   0,   0],
       [116, 104, 101, ...,   0,   0,   0],
       [111, 117, 116, ...,   0,   0,   0],
       ...,
       [116, 104,  97, ...,   0,   0,   0],
       [ 66, 101, 110, ...,   0,   0,   0],
       [105, 110, 112, ...,   0,   0,   0]], dtype=uint8), array([[ 99],
       [101],
       [117],
       [110],
       [105],
       [116],
       [ 99],
       [101],
       [ 32],
       [110],
       [108],
       [105],
       [110],
       [ 51],
       [110],
       [ 83],
       [101],
       [ 32],
       [101],
       [101],
       [111],
       [ 97],
       [ 97],
       [117],
       [121],
       [105],
       [116],
       [101],
       [110],
       [105],
       [105],
       [ 98],
       [101],
       [114],
       [ 99],
       [101],
       [ 32],
       [101],
       [102],
       [116],
       [105],
       [108],
       [ 32],
       [117],
       [109],
       [110],
       [ 97],
       [110],
       [116],
       [ 65],
       [

(array([[ 83, 101, 120, ...,   0,   0,   0],
       [ 65, 114, 101, ...,   0,   0,   0],
       [102, 111, 114, ...,   0,   0,   0],
       ...,
       [115, 101, 101, ...,   0,   0,   0],
       [ 97, 110, 100, ...,   0,   0,   0],
       [ 97,  32, 100, ...,   0,   0,   0]], dtype=uint8), array([[101],
       [104],
       [101],
       [ 78],
       [114],
       [105],
       [110],
       [110],
       [112],
       [117],
       [ 32],
       [ 32],
       [ 34],
       [ 32],
       [117],
       [ 40],
       [108],
       [ 97],
       [110],
       [105],
       [105],
       [ 99],
       [105],
       [101],
       [ 52],
       [ 44],
       [ 32],
       [105],
       [117],
       [ 44],
       [101],
       [116],
       [111],
       [114],
       [114],
       [110],
       [ 32],
       [109],
       [115],
       [116],
       [ 32],
       [108],
       [ 32],
       [ 32],
       [119],
       [108],
       [ 32],
       [ 32],
       [ 48],
       [116],
       [

KeyboardInterrupt: 